In [1]:
%load_ext autoreload
%load_ext autotime

time: 175 µs (started: 2024-10-09 15:38:02 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 536 µs (started: 2024-10-09 15:38:02 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 761 µs (started: 2024-10-09 15:38:02 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-bert-v100-node-4-v1"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/bert-v100-node-4-v1"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/bert/v100/node-4/v1/COMPACT/*.pfw.gz"


time: 631 µs (started: 2024-10-09 15:38:02 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 6.42 s (started: 2024-10-09 15:38:02 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 1min 41s (started: 2024-10-09 15:38:08 -07:00)


/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pyarrow/pandas_compat.py:456: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pyarrow/pandas_compat.py:456: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pyarrow/pandas_compat.py:456: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pyarrow/pandas_compat.py:456: DeprecationWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstanc

In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

/g/g91/izzet/projects/wisio/venv-corona/lib/python3.9/site-packages/pandas/core/frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          1097.44 seconds                                                                               │
│  I/O Time         3.25 seconds                                                                                  │
│                   ├── Read - 1.00 seconds (30.79%)                                                              │
│                   ├── Write - 0.01 seconds (0.27%)                                                              │
│                   └── Metadata - 2.31 seconds (71.23%)                                                          │
│  I/O Operations   2,175,259 ops                                                                                 │
│                   ├── Read - 72,292 ops (3.32%)                                                                 │
│                   ├── Write - 2,029 ops (0.09%)                                                                 │
│                   └── Metadata - 2,092,106 ops (96.18%)                                                         │
│  I/O Size         3.94 GiB                                                                                      │
│                   ├── Read - 3.94 GiB (99.99%)                                                                  │
│                   └── Write - 248.70 kiB (0.01%)                                                                │
│  Read Requests    4 kiB-256 kiB - 72,068 ops                                                                    │
│                   ├── <4 kiB - 43,600 ops (60.50%)                                                              │
│                   ├── 4-16 kiB - 4,960 ops (6.88%)                                                              │
│                   ├── 16-64 kiB - 7,840 ops (10.88%)                                                            │
│                   └── 64-256 kiB - 15,668 ops (21.74%)                                                          │
│  Write Requests   4 kiB-4 kiB - 2,029 ops                                                                       │
│                   └── <4 kiB - 2,029 ops (100.00%)                                                              │
│  Nodes            4 nodes                                                                                       │
│                   ├── 14200274195875936004 - 3.25 s (100.00%) - 0.98 GiB/10.56 kiB R/W (24.99/0.00%) - 543,507  │
│                   │   ops (24.99%)                                                                              │
│                   ├── 12775585946864612109 - 3.20 s (98.61%) - 0.98 GiB/10.56 kiB R/W (24.97/0.00%) - 543,513   │
│                   │   ops (24.99%)                                                                              │
│                   ├── 805730364957286384 - 3.19 s (98.20%) - 0.99 GiB/10.56 kiB R/W (25.02/0.00%) - 543,519     │
│                   │   ops (24.99%)                                                                              │
│                   └── 12604783666688308289 - 3.16 s (97.20%) - 0.99 GiB/217.01 kiB R/W (25.02/0.01%) - 544,720  │
│                       ops (25.04%)                                                                              │
│  Apps             1 app                                                                                         │
│  Processes/Ranks  549 processes                                                                                 │
│  Files            5,064 files                                                                                   │
│                   ├── Shared: 4,774 files (94.27%)                                                              │
│                   └── FPP: 290 files (5.73%)          

time: 2.89 s (started: 2024-10-09 15:39:50 -07:00)


In [ ]:
import dask.dataframe as dd

